The code needs a licence of Mosek (a free academic one works). The code was updated for Julia 1.7.3.

In my opinion, the simplest way to have a running version of Mosek for Julia, is to install the Mosek.jl package (https://github.com/MOSEK/Mosek.jl) which will install a local version of Mosek, then get an academic licence (https://www.mosek.com/products/academic-licenses/).

The list of needed packages is just below as well.

In [1]:
using LinearAlgebra, JuMP, MosekTools, ProgressMeter, Distributions, Plots

## Data Generation

In [2]:
function DataGenerationSDDP(dx ,du1, du2, T)
    A = Array{Array{Float64,2}}(undef, T); # T-Array of matrices
    B1 = Array{Array{Float64,2}}(undef, T);
    B2 = Array{Array{Float64,2}}(undef, T);
    C = Array{Array{Float64,2},1}(undef, T);
    D1 = Array{Array{Float64,2},1}(undef, T);
    D2 = Array{Array{Float64,2},1}(undef, T);
    for t = 1:T
        eps = 1.0;
        Atmp = rand(dx,dx) + 1*I(dx);
        A[t] = Atmp / (maximum(abs.(eigvals(Atmp))) + 0.01); # max eigenvalue < 1
#        A[t] = 3*Atmp / (maximum(abs(eigvals(Atmp))) + 0.01);
        B1[t] = rand(dx,du1);
        B2[t] = rand(dx,du2);
        Ctmp = rand(dx,dx);
        C[t] = Ctmp'*Ctmp;
        Dtmp = LowerTriangular(rand(du1,du1) .+ 0.01); # positive coeff on the diag. (a.s)
#        D1[t] = Dtmp'*Dtmp;
        D1[t] = I(du1);
        Dtmp = LowerTriangular(rand(du2,du2) .+ 0.01);
#        D2[t] = Dtmp'*Dtmp;
        D2[t] = Dtmp'*Dtmp + eps*I(du2);
    end
    return(A, B1, B2, C, D1, D2)
end

DataGenerationSDDP (generic function with 1 method)

In [3]:
function FixedGenerationSDDP(dx ,du1, du2, T)
    # f(x,u) = Ax + B1*u1 + B2*u2
    # c(x, u) = x^T C x + u_1^T D1 u_1 + u_2^T D2 u_2
    A = Array{Array{Float64,2}}(undef, T);
    B1 = Array{Array{Float64,2}}(undef, T);
    B2 = Array{Array{Float64,2}}(undef, T);
    C = Array{Array{Float64,2}}(undef, T);
    D1 = Array{Array{Float64,2}}(undef, T);
    D2 = Array{Array{Float64,2}}(undef, T);

    for t = 1:(T)
        eps = 0.1;
        A[t] = I(dx)*(1-eps);
#        A[t][1,1] = 2;

        B1[t] = ones(dx,du1);

        # B2[t] = eps*ones(dx,du2);
        B2[t] = ones(dx,du2);

        C[t] = 0.1*I(dx);

        D1[t] = 0.1*I(du1);

        D2[t] = 0.1*I(du2);
    end
    return(A, B1, B2, C, D1, D2)
end


FixedGenerationSDDP (generic function with 1 method)

In [4]:
"""
    From data adapted to a SDDP type problem with exactly one continuous control
    which is constrained (the rest is unconstrained), construct the associated
    data for the Qu type problem where the constrained control is discretized
    according to a vector v.
"""
function DeducedData(A, B1, B2, C, D1, D2, v, N)
    dx = length(A[1][1,:]);
    du1 = length(B1[1][1,:]);
    T = length(A);
    AQu = Matrix{Matrix{Float64}}(undef, N,T);
    BQu = Matrix{Matrix{Float64}}(undef, N,T); # One line = All control dynamic matrices for fixed switch.
    CQu = Matrix{Matrix{Float64}}(undef, N,T); # One line = All space cost matrices for a given switch.
    DQu = Matrix{Matrix{Float64}}(undef, N,T); # One line = All control cost matrices for a given switch.

    for n = 1:N
        for t = 1:T
                AQu[n,t] = [A[t] B2[t]*v[n]; zeros(1, dx) 1];
                BQu[n,t] = vcat(B1[t], zeros(1, du1));
                CQu[n,t] = [C[t] zeros(dx); zeros(1,dx) v[n]'*D2[t]*v[n]];
                DQu[n,t] = D1[t];
        end
    end
    return(AQu, BQu, CQu, DQu)
end


DeducedData

## I] SDDP

In [33]:
"""
    Return opt. control and a cut at B_t( V_{t+1}^k )(x_t^k) for a given t.
    B1 matrix in (unconstrained) u of the dynamics
    B2 matrix in (constrained) u of the dynamics
"""
function BellmanSDDP(A, B1, B2, C, D1, D2, t, xt, Vtplus1, a, b; BOX = 100)
    du1 = length(D1[t][1,:]); # Dimension of the unconstrained control space
    du2 = length(D2[t][1,:]); # Dimension of the constrained control space
    dx = length(xt); # We guess the dimension of the state space, dx.
    l = length(Vtplus1); # We guess the current number of cuts.
    Pb = Model(optimizer_with_attributes(Mosek.Optimizer, "QUIET" => true)); # Solver that solves QP.
    @variable(Pb, a <= u2[1:du2] <= b)
    @variable(Pb, x[1:dx])
    @variable(Pb, u1[1:du1])
    @constraint(Pb, test[i=1:dx], x[i] - xt[i] == 0)
    @constraint(Pb, box_x[i=1:dx], -BOX <= x[i] <= BOX) # big Box constraint to enter the framework of SDDP
    @constraint(Pb, box_u[j=1:du1], -BOX <= u1[j] <= BOX)
    if t <= T-2
            @variable(Pb, theta >= 0)
        if l >= 2
            @constraint(Pb, cuts[i=2:l], theta >= Vtplus1[i][1]'*(
                A[t]*x + B1[t]*u1 + B2[t]*u2) + Vtplus1[i][2])
        end
        @objective(Pb, Min, x'*C[t]*x + u1'*D1[t]*u1 + u2'*D2[t]*u2 + theta)

    else # t = T-1 and we compute B_{T-1}( V_T )(x_{T-1})
        @objective(Pb, Min, x'*C[t]*x + u1'*D1[t]*u1 + u2'*D2[t]*u2 +
            (A[t]*x + B1[t]*u1 + B2[t]*u2)'*Vtplus1*(A[t]*x + B1[t]*u1
            + B2[t]*u2))
    end
    optimize!(Pb);
    return(value.(u1), value.(u2), dual.(test),
        objective_value(Pb)- sum(dual.(test).*xt))
end

BellmanSDDP

In [6]:
"""
One iteration of SDDP in the LQ framework.
A, B1, B2, C, D1, D2 are problem data (array of matrices for the
dynamics and controls)
Vtplus1 is the set of functions building the current approximations.
x0 is the starting point at time t=0.
""" 
function SDDP(A, B1, B2, C, D1, D2, Vt, x0, a, b; constraint = false)
    # We deduce the dimensions of the problems from the given data
    T = length(A); # We guess T.
    dx = length(A[1][1,:]); # We guess the dimension of the state space, dx.
    du1 = length(B1[1][1,:]); # Dimension of the unconstrained control space.
    du2 = length(B2[1][1,:]); # Dimension of the constrained control space.
    NewCutSlopes = Array{Array{Float64,1}}(undef, T-1); # Slopes of the added cuts.
    NewCutValuesAt0 = Array{Float64}(undef, T-1); # Values of the added cuts at 0.

    # Forward pass
    x = zeros(dx, T); # Contains an optimal trajectory for the current approx.
    x[:, 1] = x0; # starting from x0.
    for t = 1:(T-1)
        utmp1, utmp2 = BellmanSDDP(A, B1, B2, C, D1, D2, t, x[:,t], Vt[t+1],
            a, b);
        x[:,t+1] = A[t]*x[:,t] + B1[t]*utmp1 + B2[t]*utmp2;
        # We add some state constraints for the first few Iterations
        # in order to avoid numerical issues (too high first few
        # values)
        if constraint == true
            for i = 1:dx
                if abs(x[i, t+1]) > 50
                    x[i,t+1] = sign(x[i,t+1])*50
                end
            end
        end
    end
    # Backward pass
    for t = (T-1):-1:1
        tmp1, tmp2, NewCutSlopes[t], NewCutValuesAt0[t] = BellmanSDDP(A,
         B1, B2, C, D1, D2, t, x[:,t], Vt[t+1], a, b);
    end

    return(NewCutSlopes, NewCutValuesAt0, x)
end


SDDP

## II] MinPlus

In [7]:
"""
Computes the Bellman image of a pure strict convex quadratic form V given the
 data matrices by A,B,C,D in the case where we compute it through
 Riccati's formula.

Linear dynamic: Ax + Bu,
Pure convex quadratic costs: x^T C x + u^T D u,
Pure final quadratic cost: x^T Psi x,
Pure stric convex quadratic form: x^T M x
"""

function Riccati(A,B,C,D, M)
    d = length(A[:,1]);
    return(A'*M*inv(I(d)+B*inv(D)*(B')*M)*A + C)
end


Riccati (generic function with 1 method)

In [8]:
"""
    Draw uniformly T points on the euclidean sphere of R^dx, with dx > 1.

    Needs to use the package Distributions.
"""
function UniformSphere(dx,T)
    tmp = rand(Normal(),dx,T);
    v = 1 ./ sqrt.(sum(tmp.^2, dims=1));
    return(tmp*Diagonal(reshape(v, length(v))))
end


UniformSphere

In [9]:
"""
MinPlus algorithm, finite horizon, discrete time, finite number of switching control.


Linear dynamic: Ax + Bu,
Pure convex quadratic costs: x^T C x + u^T D u,
Current approximations : F, one line of array of matrices.

Returns an array of T values (NewV) and one line of matrices (NewF).
"""

# C'EST UNE PROCEDURE

function Qu(A, B, C, D, Func, SwitchPreviousQuad)
    T = length(A[1,:]); # We guess T.
    dx = length(A[1,1][:,1]); # We guess the dimension of the state space, dx.
    N = length(A[:,1]); # We guess the number of switching contorls, N.
    NewF = Array{Any}(undef, T); # New quad forms that will be added (or not) B(q_i)
    NewSwitches = zeros(Int64, T-1); # The optimal switches associated.
    Index = zeros(Int64, T-1); # Index i of B(q_i)(x) (opt quad forms at t+1)

    UniformDraw = UniformSphere(dx, T);
    # We start at time t = T
    TailleApproxActuelle = length(Func[T]);
    DrawActuel = UniformDraw[:,T];
    ValApproxActuelle = zeros(TailleApproxActuelle);
    for indiceCollection = 1:TailleApproxActuelle
        ValApproxActuelle[indiceCollection] = (DrawActuel'*Func[T][indiceCollection]*DrawActuel)[1];
    end
    index = findmin(ValApproxActuelle)[2];
    NewF[T] = Func[T][index];

    # We update backward in time, using that we updated (or not) at time t+1
    for t = T-1:-1:1
        if (NewF[t+1] in Func[t+1]) == false
            Func[t+1] = vcat(Func[t+1],[NewF[t+1]]);
            SwitchPreviousQuad[t+1] = hcat(SwitchPreviousQuad[t+1],
             [NewSwitches[t+1], Index[t+1]]);
        end
        TailleApproxActuelle = length(Func[t+1]);
        DrawActuel = UniformDraw[:,t];
        ValImageApproxActuelle = zeros(TailleApproxActuelle);
        ImageApproxActuelle = Array{Matrix{Float64}}(undef, TailleApproxActuelle);
        SwitchingIndexes = zeros(Int64, TailleApproxActuelle);
        for indiceCollection = 1:TailleApproxActuelle
            # Contient image de l'approx actuelle par Bellman pour chaque switch (une fonction)
            tampon = Array{Any}(undef, N);
            # Contient valeurs au pts random de l'approx actuelle par Bellman
            # pour chaque switch.
            ValeurTmp = zeros(N);
            for switch = 1:N
                tampon[switch] = Riccati(A[switch,t], B[switch,t],
                C[switch,t], D[switch,t], Func[t+1][indiceCollection]);
                ValeurTmp[switch] = (DrawActuel'*tampon[switch]*DrawActuel)[1];
            end
            # On trouve meilleur switch pour la fct au temps t+1 fixee
            ValImageApproxActuelle[indiceCollection], SwitchingIndexes[indiceCollection] = findmin(ValeurTmp);
            # Et son image par Bellman
            ImageApproxActuelle[indiceCollection] = tampon[SwitchingIndexes[indiceCollection]];
        end
        # On trouve l'indice Index[t] de la meilleure fonction
        Index[t] = findmin(ValImageApproxActuelle)[2];
        NewSwitches[t] = SwitchingIndexes[Index[t]];
        NewF[t] = ImageApproxActuelle[Index[t]];
    end

    # We update at time 0 (encoded by 1)
    if (NewF[1] in Func[1]) == false
        Func[1] = vcat(Func[1],[NewF[1]]);
        SwitchPreviousQuad[1] = hcat(SwitchPreviousQuad[1],
         [NewSwitches[1], Index[1]]);
    end
end


Qu (generic function with 1 method)

## MinPlus_SDDP

In [55]:
function QuSDDPtogether(N, dx, du1, T, time, x0, itermax, a, b)
    println("-----------")
    # -----
    # Data Generation
    # -----
    # A, B1, B2, C, D1, D2 = DataGenerationSDDP(dx ,du1, du2, T);
     A, B1, B2, C, D1, D2 = FixedGenerationSDDP(dx, du1, du2, T);
    # -----
    # SDDP Initizalization
    # -----
    Vt = Array{Any}(undef, T); # Will contain the cuts building our approx.
    MatTmp =  rand(dx,dx);
    eps = 1.0;
    timerSDDP = zeros(itermax)
#    Vt[T] = eps*I(dx) + MatTmp'*MatTmp; # cost function at time T is known
    Vt[T] = I(dx);
    Trajectories = Array{Any}(undef, itermax+1);
    traj = Matrix(undef, dx, T);
    traj[:, 1] = x0;
    for t = 1:(T-1)
        # traj[:, t+1] = A[t]*traj[:, t];
        traj[:, t+1] = x0;
    end
    Trajectories[1] = traj;
    for t = (T-1):-1:1
        Vt[t] = [[zeros(dx), -Inf]];
    end

    # -----
    # Discretization of the constrained continuous control with N "uniform" points.
    # -----
    v = zeros(Float64, N);
    v[1] = a;
    for i = 1:(N-1)
        v[i+1] = a + i*((b-a)/(N-1));
    end

    # -----
    # Deduced data for Qu A, B, C, D, F, SwitchPreviousQuad
    # -----
    AQu, BQu, CQu, DQu = DeducedData(A, B1, B2, C, D1, D2, v, N)

    # -----
    # Qu initialization
    # -----
    F = Array{Any}(undef, T);
    SwitchPreviousQuad = Array{Any}(undef, T);# Array of switches & index of next quad form
    timerQu = zeros(itermax); # We save the time spent at each step.
    TailleF0 = zeros(Int64, itermax); # length of the final set F0 at each step.
    TailleF0[1] = 1;
    #M = Inf*I(dx+1);
    M = 500*I(dx+1) # FIND TRUE SEQUENCE OF CONStaNTs LIKE IN THE PROPOSITION
    for t = 1:(T-1)
        F[t] = [M]; # Technical issue, those elements don't serve any purpose.
    end
    F[T] = [[Vt[T] zeros(dx); zeros(1, dx) 0]]; # Final cost function put into R^(n+1)
    # by the identity mapping.
    for t = 1:T
        SwitchPreviousQuad[t] = ["Associated switch", "Index of quad at t+1"];
    end
    # -----
    # QuSDDP loop
    # -----
    println("SDDP-MinPlus Loop and plot generation")
    prog = Progress(itermax);
    ImprovementIndexes = Array{Any}(undef, T);
    for t = 1:T
        ImprovementIndexes[t] = [1];
    end
    CurrentSize = zeros(Int64, T);
    anim = @animate for k = 1:itermax
        # Qu Loop & additional stuff for the plots
        for t = 1:T
            CurrentSize[t] = length(F[t]);
        end
        timerQu[k] = @elapsed begin
            Qu(AQu, BQu, CQu, DQu, F, SwitchPreviousQuad);
            for t = 1:T
                if k != 1 && CurrentSize[t] != length(F[t])
                    ImprovementIndexes[t] = vcat(ImprovementIndexes[t], k);
                end
            end
            TailleF0[k] = length(F[1]);
        end
        # SDDP Loop & current optimal trajectory for the plots
        timerSDDP[k] = @elapsed begin
            # We add some state constraints for the first few Iterations
            # in order to avoid numerical issues (too high first few
            # values)
            bool = false;
            if k <= 2*dx
                bool = true;
            end
            NewCutSlopes, NewCutValuesAt0, traj = SDDP(A, B1, B2, C, D1, D2, Vt, x0, a, b; constraint = bool);
            Trajectories[k+1] = traj;
            for t = 1:(T-1)
                Vt[t] = hcat(Vt[t], [[NewCutSlopes[t], NewCutValuesAt0[t]]]);
            end
        end
        # Plotting part, first a "gap plot", then heatmap&surface at given t.
        # Gap plot
        p1, p2 = PlotGap(traj, F, Vt, N);
        # Heatmap
        coord = [1, 2];
        perturb = 5;
        square = [
            traj[:, time][coord[1]]- perturb, traj[:, time][coord[1]] + perturb,
            traj[:, time][coord[2]] - perturb, traj[:, time][coord[2]] + perturb
            ];
        p3 = SuperHeatmap(traj, F, Vt, time, square, coord);
        # # Surface plot
        # p4 = SurfacePlot(x, F, Vt, coord);
        l = @layout [ grid(2,1) a{0.5w} ]
        plot(p1, p2, p3, layout = l)
        # println(traj)
        next!(prog)
    end
    gif(anim, "Gap_Plots.gif", fps = 2)

    # Timers

    plotTimerQu = plot(1:1:itermax, cumsum(timerQu), title = "Time spent",
    label = "MinPlus", legend = :topleft, xlabel = "Iterations",
        ylabel = "Cummulative time spent (s)");
    plotTimerSDDP = plot!(1:1:itermax, cumsum(timerSDDP), label = "SDDP");
    titleTimers = string("Time_Plots_", N,".pdf")
    savefig(plotTimerSDDP , titleTimers )
    #sizesOfF = zeros(Int64, T-1);
    #for t = 1:T-1
    #    sizesOfF[t] = length(F[t]);
    #end
    
#     # Saving Data
#     save("Data.jld",
#         "Set building Qu approximations", F,
#         "Set building SDDP approximations", Vt,
#         "Improvement Indexes", ImprovementIndexes,
#         "Sizes of F", sizesOfF,
#         "Number of discretizations", N,
#         "Horizon T", T,
#         "Lower bound on the constrained control", a,
#         "Upper bound on the constrained control", b,
#         "Dimension of x", dx,
#          "Max iterations", itermax,
#         "Dimension of unconstrained controls", du1,
#         "Initial point", x0,
#         "Specific chosen time for plots", time,
#         "Timer SDDP", timerSDDP,
#         "Timer Qu", timerQu,
#         "Switching sequences", SwitchPreviousQuad,
#         "Matrices A", A,
#         "Matrices B1", B1,
#         "Matrices B2", B2,
#         "Matrices C", C,
#         "Matrices D1", D1,
#         "Matrices D2", D2,
#         "Optimal trajectories for current approximations", Trajectories)
end


QuSDDPtogether (generic function with 1 method)

## Plotting functions


In [51]:
"""
INPUT
x current optimal trajectory for the current outer approximation of the value
    functions.
F array of size T. Contains in each cell an array of matrices which are
associated with the infimum (of pure quad forms) building Vt in Qu's algorithm.
Vt array of size T. Contains in each celle a couple [slope, value_at_0] which
are associated with the supremum (of affine cuts) building Vt in SDDP algo.

OUTPUT
A plot of the value of the approximations at points x_0^k, ..., x_T^k.
"""
function PlotGap(x, F, Vt, N)
    valQu = zeros(Float64, T-1);
    valSDDP = zeros(Float64, T-1);
    Gaps = zeros(Float64, T-1);
    for t = 1:(T-1)
        valQu[t] = vcat(x[:,t], 1)'*F[t][2]*vcat(x[:,t], 1);
        for i = 2:(length(F[t]) - 1)
            valQu[t] = min(vcat(x[:,t],1)'*F[t][i+1]*vcat(x[:,t],1), valQu[t])[1];
        end
        #println("valQu at t = ", t, "is equal to ", valQu[t])
        valSDDP[t] = dot(Vt[t][2][1], x[:,t]) + Vt[t][2][2];
        for i = 2:(length(Vt[t]) - 1)
            valSDDP[t] = max(valSDDP[t],
                dot(Vt[t][i+1][1], x[:,t]) + Vt[t][i+1][2])[1];
        end
        Gaps[t] = (valQu[t] - valSDDP[t])
    end
    titlestring = string("Iteration ", length(Vt[1]) - 1, ", N = ", N)
    plotValeurs = plot(1:1:(T-1), (valQu), label = "MinPlus", w = 3, legend = :best, 
        title = titlestring, xlabel = "Time step (t)", ylabel = "Values");
    plotValeurs = plot!(1:1:(T-1), (valSDDP), label = "SDDP", w = 3);
    plotGaps = plot(1:1:(T-1), Gaps, leg = false, title =
            "Absolute gaps", w = 4);
    # Temporaire
    if length(Vt[3]) == 21
        titlespecial = string("Ex2_PlotGapSpecifique_", N,".pdf")
        savefig(plotValeurs, titlespecial)
    end
#      if length(Vt[3]) == 19
#          savefig(plotValeurs, "PlotGapSpecifique18.pdf")
#      end
    
#      if length(Vt[3]) == 61
#          savefig(plotValeurs, "PlotGapSpecifique60.pdf")
#      end 
    return(plotValeurs, plotGaps)
end



PlotGap

In [12]:
"""
INPUT : The current optimal trajectory for the current SDDP approximation, traj.
The sets F and Vt building respectively the upper and lower bounds.
A specific time.
An array, square = [a,b,c,d], caracterizing the square on which we will plot.
Two coordinates on which we will project.

OUTPUT : A heatmap of (a projection on a 2-diemnsional v.space of) Vt where
there's also the current x_t^k.
"""

function SuperHeatmap(traj, F, Vt, time, square, coord)
    stepsize = 0.5;
    x1 = square[1]:stepsize:square[2];
    x2 = square[3]:stepsize:square[4];
    x = copy(traj[:, time]);
    valQu = zeros( Float64, (length(x1),length(x2)) );
    valSDDP = zeros( Float64, (length(x1),length(x2)) );

    for i = 1:length(x1)
        for j = 1:length(x2)
            x[coord[1]] = x1[i];
            x[coord[2]] = x2[j];

            valQu[i,j] = vcat(x, 1)'*F[time][2]*vcat(x, 1);
            for l = 2:(length(F[time]) - 1)
                valQu[i,j] = min(vcat(x, 1)'*F[time][l+1]*vcat(x, 1),
                    valQu[i,j])[1];
            end

            valSDDP[i,j] = dot(Vt[time][2][1], x) + Vt[time][2][2];
            for l = 2:(length(Vt[time]) - 1)
                valSDDP[i,j] = max(valSDDP[i,j],
                    dot(Vt[time][l+1][1], x) + Vt[time][l+1][2])[1];
            end
        end
    end
    superHeatmap = heatmap(x1, x2, valQu - valSDDP, title =
        "2D log-gap projection");
    return(superHeatmap)
end


SuperHeatmap (generic function with 1 method)

## Test

In [43]:
# This is Example 1 in [Akian, Chancelier, Tran, A stochastic algorithm for deterministic multistage optimization problems]

N = 5; # Cardinal of the set of discrete controls.
dx = 100; # tate space dimension.
du1 = 5; # Dimension of the unconstrained control space.
T = 21; # Finite horizon setup. One less (there s no 0) than the theorical T.
time = 3; # We will later plot the approximate value functions at this t;
x0 = 0.2*ones(dx);
itermax = 60;
a = 1 ; b = 5; du2 = 1; # Contraintes sur UN contrôle continu, du2 = 1.
 

In [49]:
QuSDDPtogether(N, dx, du1, T, time, x0, itermax, a, b)

-----------
SDDP-MinPlus Loop and plot generation


Progress: 100%|█████████████████████████████████████████| Time: 0:00:56
┌ Info: Saved animation to 
│   fn = /home/tranbe/Dropbox/these/jpc-ma-bt/Code/TDP_Deterministe/Gap_Plots.gif
└ @ Plots /home/tranbe/.julia/packages/Plots/MzlNY/src/animation.jl:130


In [56]:
# This is Example 2 in [Akian, Chancelier, Tran, A stochastic algorithm for deterministic multistage optimization problems]

dx = 25; # state space dimension.
du1 = 5; # Dimension of the unconstrained control space.
T = 21; # Finite horizon setup. One less (there s no 0) than the theorical T.
time = 3; # We will later plot the approximate value functions at this t;
x0 = 0.2*ones(dx);
itermax = 60;
a = -3 ; b = 5; du2 = 1; # Contraintes sur UN contrôle continu, du2 = 1.
 
for N in [5,20,100]
    QuSDDPtogether(N, dx, du1, T, time, x0, itermax, a, b)
end

-----------
SDDP-MinPlus Loop and plot generation


Progress: 100%|█████████████████████████████████████████| Time: 0:00:18


-----------
SDDP-MinPlus Loop and plot generation


┌ Info: Saved animation to 
│   fn = /home/tranbe/Dropbox/these/jpc-ma-bt/Code/TDP_Deterministe/Gap_Plots.gif
└ @ Plots /home/tranbe/.julia/packages/Plots/MzlNY/src/animation.jl:130
Progress: 100%|█████████████████████████████████████████| Time: 0:00:55


-----------


┌ Info: Saved animation to 
│   fn = /home/tranbe/Dropbox/these/jpc-ma-bt/Code/TDP_Deterministe/Gap_Plots.gif
└ @ Plots /home/tranbe/.julia/packages/Plots/MzlNY/src/animation.jl:130


SDDP-MinPlus Loop and plot generation


Progress: 100%|█████████████████████████████████████████| Time: 0:01:51
┌ Info: Saved animation to 
│   fn = /home/tranbe/Dropbox/these/jpc-ma-bt/Code/TDP_Deterministe/Gap_Plots.gif
└ @ Plots /home/tranbe/.julia/packages/Plots/MzlNY/src/animation.jl:130
